# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
df = pd.read_csv('Orders.zip')
df.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [3]:
df.dtypes

Unnamed: 0        int64
InvoiceNo         int64
StockCode        object
year              int64
month             int64
day               int64
hour              int64
Description      object
Quantity          int64
InvoiceDate      object
UnitPrice       float64
CustomerID        int64
Country          object
amount_spent    float64
dtype: object

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# your code here
aggregated_spent = df.groupby(['CustomerID'])['amount_spent'].agg('sum').reset_index()
aggregated_spent

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [5]:
print("We have ", aggregated_spent['CustomerID'].count(), " customers with purchase value")

We have  4339  customers with purchase value


In [6]:
#VIP = 0.95 quantile  - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html
aggregated_spent['amount_spent'].quantile(.95)

VIP_customers = aggregated_spent.loc[aggregated_spent['amount_spent'] >= aggregated_spent['amount_spent'].quantile(.95)].sort_values(by=['amount_spent'],ascending=False).reset_index(drop=True)
VIP_customers

,CustomerID,amount_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06
...,...,...
212,14505,5905.64
213,14415,5900.28
214,14944,5900.25
215,13209,5891.61


In [7]:
print("We have ", VIP_customers['CustomerID'].count(), " VIP Customers and their minimum spent amount is: ", int(VIP_customers['amount_spent'].min()) )



We have  217  VIP Customers and their minimum spent amount is:  5870


In [8]:
Preferred_customers_VIP = aggregated_spent.loc[aggregated_spent['amount_spent'] >= aggregated_spent['amount_spent'].quantile(.75)].sort_values(by=['amount_spent'],ascending=False).reset_index(drop=True)
Preferred_customers_VIP

,CustomerID,amount_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06
...,...,...
1080,17656,1674.69
1081,16115,1667.97
1082,12912,1662.30
1083,12530,1662.28


In [9]:
#Removing the VIP customers to single out the Preferred
Preferred_customers = Preferred_customers_VIP.drop(Preferred_customers_VIP.index[0:217])
Preferred_customers

,CustomerID,amount_spent
217,13050,5836.86
218,12720,5781.73
219,15218,5756.89
220,17686,5739.46
221,13178,5725.47
...,...,...
1080,17656,1674.69
1081,16115,1667.97
1082,12912,1662.30
1083,12530,1662.28


In [10]:
print("We have ", Preferred_customers['CustomerID'].count(), " Preferred Customers and their minimum spent amount is: ", int(Preferred_customers['amount_spent'].min()) )


We have  868  Preferred Customers and their minimum spent amount is:  1661


In [11]:

conditions = [
    (aggregated_spent['amount_spent'] >= aggregated_spent['amount_spent'].quantile(.95)),
    ((aggregated_spent['amount_spent'] <  aggregated_spent['amount_spent'].quantile(.95)) & (aggregated_spent['amount_spent'] > aggregated_spent['amount_spent'].quantile(.75))),
    (aggregated_spent['amount_spent'] < aggregated_spent['amount_spent'].quantile(.75))
     ]   
     
results = ['VIP', 'Preferred', 'Normal']   
    
aggregated_spent['Customer Type'] = np.select(conditions, results, default='-')
aggregated_spent    

,CustomerID,amount_spent,Customer Type
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Normal
...,...,...,...
4334,18280,180.60,Normal
4335,18281,80.82,Normal
4336,18282,178.05,Normal
4337,18283,2094.88,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [12]:
# your code here
#(1)adding the customer type to each customer ID of the main table
df_merge = pd.merge(df, aggregated_spent, on="CustomerID")
df_merge

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,amount_spent_y,Customer Type
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541801,581578,22993,2011,12,5,12,set of 4 pantry jelly moulds,12,2011-12-09 12:16:00,1.25,12713,Germany,15.00,848.55,Normal
397920,541802,581578,22907,2011,12,5,12,pack of 20 napkins pantry design,12,2011-12-09 12:16:00,0.85,12713,Germany,10.20,848.55,Normal
397921,541803,581578,22908,2011,12,5,12,pack of 20 napkins red apples,12,2011-12-09 12:16:00,0.85,12713,Germany,10.20,848.55,Normal
397922,541804,581578,23215,2011,12,5,12,jingle bell heart antique silver,12,2011-12-09 12:16:00,2.08,12713,Germany,24.96,848.55,Normal


In [13]:
country_cust_type = df_merge.groupby(['Country'])['Customer Type'].value_counts()
country_cust_type
#antes do groupby fazer uma lambda que isola os VIPS, filter

Country               Customer Type
Australia             VIP                 898
                      Normal              157
                      Preferred           130
Austria               Normal              240
                      Preferred           158
                                        ...  
United Arab Emirates  Normal               68
United Kingdom        Preferred        137450
                      Normal           132710
                      VIP               84185
Unspecified           Normal              244
Name: Customer Type, Length: 76, dtype: int64

In [31]:
#filtering only the VIP customers to then rank countries
VIP_only = df_merge.loc[df_merge['Customer Type'] == 'VIP']
VIP_rank = VIP_only.groupby(['Country'])['Customer Type'].count().sort_values(ascending=False)
VIP_rank

Country
United Kingdom     84185
EIRE                7077
France              3290
Germany             3127
Netherlands         2080
Australia            898
Portugal             681
Switzerland          594
Spain                511
Norway               420
Channel Islands      364
Finland              294
Cyprus               248
Singapore            222
Japan                205
Sweden               198
Belgium               54
Denmark               36
Name: Customer Type, dtype: int64

In [32]:
most_VIPs = VIP_rank.head(1)
most_VIPs

Country
United Kingdom    84185
Name: Customer Type, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [39]:
#filtering only the VIP and Preferred customers to then rank countries
VIP_Pref = df_merge.loc[df_merge['Customer Type'] != 'Normal']
VIP_Pref

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,amount_spent_y,Customer Type
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397647,533738,581105,22961,2011,12,3,12,jam making set printed,620,2011-12-07 12:14:00,1.38,16000,United Kingdom,855.60,12393.70,VIP
397648,533739,581105,23552,2011,12,3,12,bicycle puncture repair kit,620,2011-12-07 12:14:00,1.86,16000,United Kingdom,1153.20,12393.70,VIP
397649,533741,581107,23461,2011,12,3,12,sweetheart bird house,620,2011-12-07 12:15:00,3.30,16000,United Kingdom,2046.00,12393.70,VIP
397650,533751,581110,23486,2011,12,3,12,antique heart shelf unit,150,2011-12-07 12:17:00,13.35,16000,United Kingdom,2002.50,12393.70,VIP


In [40]:
VIP_Pref2 = VIP_Pref.groupby(['Country'])['Customer Type'].count().sort_values(ascending=False)
VIP_Pref2

Country
United Kingdom     221635
Germany              7349
EIRE                 7238
France               6301
Netherlands          2080
Spain                1569
Belgium              1557
Switzerland          1370
Portugal             1093
Australia            1028
Norway               1028
Channel Islands       589
Italy                 507
Finland               504
Cyprus                451
Sweden                273
Japan                 272
Singapore             222
Denmark               217
Israel                214
Iceland               182
Austria               158
Poland                149
Canada                135
Malta                  67
Lebanon                45
Greece                 32
Name: Customer Type, dtype: int64

In [42]:
most_VIP_Pref = VIP_Pref2.head(1)
most_VIP_Pref

Country
United Kingdom    221635
Name: Customer Type, dtype: int64